In [31]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.features import geometry_mask, rasterize
from rasterio.transform import from_origin
from rasterio.mask import mask
from rasterio.plot import show

import matplotlib.pyplot as plt

In [34]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.features import geometry_mask, rasterize
from rasterio.transform import from_origin
from rasterio.mask import mask

# Read the CSV and California shapefile
csv_filepath = 'raw_data/avg_AQI_2016_Q1.csv'
df = pd.read_csv(csv_filepath)

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude), crs="EPSG:32633")
gdf['geometry'] = gdf['geometry'].buffer(0.05)  # Buffer the points

california_border = gpd.read_file("HUC8_CA_PFAS_GTruth.shp")
california_border.crs = "EPSG:4269"

# Determine raster extent and pixel size
xmin, ymin, xmax, ymax = california_border.total_bounds
pixel_size = 0.01  # Adjust as needed
width = int((xmax - xmin) / pixel_size)
height = int((ymax - ymin) / pixel_size)
transform = from_origin(xmin, ymax, pixel_size, pixel_size)

# Create raster for buffer zones with AQI values
shapes_with_aqi = [(geom, value) for geom, value in zip(gdf['geometry'], gdf['avg_AQI'])]

buffered_raster = rasterize(shapes_with_aqi, out_shape=(height, width), transform=transform, fill=0, dtype=rasterio.float32)

# Create raster for California border
california_raster = np.zeros((height, width), dtype=np.uint8)
california_raster_mask = geometry_mask(california_border['geometry'], transform=transform, invert=True, out_shape=(height, width))
california_raster[california_raster_mask] = 1  # Set pixels inside California border to 1

# Combine the two rasters by taking the max value (AQI values will override the border)
combined_raster = np.maximum(buffered_raster, california_raster)

combined_raster_path = 'test_out.tif'
with rasterio.open(
    combined_raster_path, 'w',
    driver='GTiff',
    height=combined_raster.shape[0],
    width=combined_raster.shape[1],
    count=1,
    dtype=rasterio.float32,
    crs="EPSG:4269",
    transform=transform,
) as dst:
    dst.write(combined_raster, 1)

# Mask out values outside California
ca_shapes = [feature["geometry"] for feature in california_border.__geo_interface__["features"]]
with rasterio.open('test_out.tif', 'r+') as src:
    # Read the entire raster
    data = src.read(1)

    # Generate mask
    mask_array, _ = mask(src, ca_shapes, crop=False)

    # Set values outside the mask to NoData
    nodata_value = src.nodata if src.nodata is not None else -60
    data[mask_array[0] == False] = nodata_value

    # Update metadata to include NoData value
    out_meta = src.meta.copy()
    out_meta.update({"nodata": nodata_value})

    src.write(data, 1)

In [35]:

filepath = 'test_out.tif'
with rasterio.open(filepath) as src:
    # Get resolution
    resolution = src.res
    # Get bounding box (min/max longitude and latitude)
    bounds = src.bounds
    min_lon, max_lon = bounds.left, bounds.right
    min_lat, max_lat = bounds.bottom, bounds.top
    # Get pixel dimensions
    width, height = src.width, src.height
    # Print information
    print(f"File: {filepath}")
    print(f"  Resolution: {resolution}")
    print(f"  Bounding Box: ({min_lon}, {max_lon}, {min_lat}, {max_lat})")
    print(f"  Pixel Dimensions: {width} x {height}")

File: test_out.tif
  Resolution: (0.01, 0.01)
  Bounding Box: (-124.34811344799999, -114.13811344799998, 32.54367419300005, 42.00367419300005)
  Pixel Dimensions: 1021 x 946


In [31]:
import rasterio
from rasterio.enums import Resampling

# Open the source TIFF file
with rasterio.open('test_out.tif') as src:
    # Calculate new shape for doubling
    new_width = src.width * 10
    new_height = src.height * 10

    # Calculate new transform (to adjust the resolution)
    new_transform = src.transform * src.transform.scale(
        (src.width / new_width),  # scaling in the X direction
        (src.height / new_height)  # scaling in the Y direction
    )
    
    # Read and resample the data to the new shape
    data = src.read(
        out_shape=(src.count, new_height, new_width),
        resampling=Resampling.bilinear  # you can use other resampling methods if needed
    )
    
    # Update metadata
    profile = src.profile
    profile.update({
        'height': new_height,
        'width': new_width,
        'transform': new_transform,
        'driver': 'GTiff'
    })

    # Write the output to a new TIFF file
    with rasterio.open('output_doubled.tiff', 'w', **profile) as dst:
        dst.write(data)
